# HuggingFace Embeddings Tutorial

이 튜토리얼에서는 LangChain과 HuggingFace를 활용한 다양한 임베딩 방법을 다룹니다:
- HuggingFace Endpoint Embeddings
- HuggingFace Local Embeddings
- BGE-M3 임베딩
- FlagEmbedding (Dense, Sparse, Multi-Vector)

## 환경 설정

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os
import warnings

# API KEY 정보로드
load_dotenv()

# 경고 무시
warnings.filterwarnings("ignore")

# ./cache/ 경로에 다운로드 받도록 설정
os.environ["HF_HOME"] = "./cache/"

## 테스트 데이터 준비

In [2]:
# 다국어 테스트 텍스트
texts = [
    "안녕, 만나서 반가워.",
    "LangChain simplifies the process of building applications with large language models",
    "랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. ",
    "LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]

## 1. HuggingFace Endpoint Embedding

`HuggingFaceEndpointEmbeddings`는 HuggingFace의 추론 API를 사용하여 임베딩을 생성합니다. 로컬에 모델을 다운로드할 필요 없이 API를 통해 임베딩을 계산할 수 있습니다.

In [3]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)

### Document 임베딩 생성

In [4]:
# Document Embedding 수행
embedded_documents = hf_embeddings.embed_documents(texts)

print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

Model: 		intfloat/multilingual-e5-large-instruct
Dimension: 	1024


### Query 임베딩 생성

In [5]:
# Query Embedding 수행
embedded_query = hf_embeddings.embed_query("LangChain 에 대해서 알려주세요.")
print(f"Query embedding dimension: {len(embedded_query)}")

Query embedding dimension: 1024


## 유사도 계산

### 벡터 내적을 통한 유사도 계산

벡터 내적(dot product)을 사용하여 유사도를 계산합니다.

**수학적 배경:**

1. **벡터 내적 정의**
   $$ \mathbf{a} \cdot \mathbf{b} = \sum_{i=1}^{n} a_i b_i $$

2. **코사인 유사도와의 관계**
   $$ \mathbf{a} \cdot \mathbf{b} = \|\mathbf{a}\| \|\mathbf{b}\| \cos \theta $$

3. **유사도 해석**
   - 내적 값이 클수록 두 벡터가 유사
   - 두 벡터가 같은 방향을 가리킬수록 높은 유사도

In [6]:
import numpy as np

# 질문(embedded_query): LangChain 에 대해서 알려주세요.
similarities = np.array(embedded_query) @ np.array(embedded_documents).T
print(f"유사도 점수: {similarities.round(3)}")

유사도 점수: [0.842  0.865  0.865  0.896  0.768]


In [7]:
# 유사도 순으로 정렬
sorted_idx = similarities.argsort()[::-1]

print("[Query] LangChain 에 대해서 알려주세요.\n====================================")
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {texts[idx]}")

[Query] LangChain 에 대해서 알려주세요.
[0] LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.
[1] LangChain simplifies the process of building applications with large language models
[2] 랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. 
[3] 안녕, 만나서 반가워.
[4] Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.


## 2. HuggingFace Local Embeddings

`HuggingFaceEmbeddings`는 모델을 로컬에 다운로드하여 사용합니다. API 호출 없이 로컬에서 임베딩을 생성할 수 있어 더 빠른 처리가 가능합니다.

### 사용 가능한 모델들
- [intfloat/multilingual-e5-large-instruct](https://huggingface.co/intfloat/multilingual-e5-large-instruct)
- [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large)

In [8]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

hf_local_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"},  # cuda, cpu, mps(Mac)
    encode_kwargs={"normalize_embeddings": True},  # 정규화된 임베딩 사용
)

In [9]:
# Document Embedding
embedded_documents_local = hf_local_embeddings.embed_documents(texts)

print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents_local[0])}")

Model: 		intfloat/multilingual-e5-large-instruct
Dimension: 	1024


## 3. BGE-M3 임베딩

BGE-M3는 다국어 지원이 우수한 임베딩 모델입니다.

### 디바이스 옵션
- `{"device": "cpu"}`: CPU를 사용하여 임베딩 계산 (모든 사용자)
- `{"device": "cuda"}`: GPU를 사용하여 임베딩 계산 (CUDA 설치 필요)
- `{"device": "mps"}`: Apple Silicon GPU 사용 (Mac 사용자)

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

bge_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, 
    model_kwargs=model_kwargs, 
    encode_kwargs=encode_kwargs
)

In [11]:
# Document Embedding
bge_embedded_documents = bge_embeddings.embed_documents(texts)

print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(bge_embedded_documents[0])}")

Model: 		BAAI/bge-m3
Dimension: 	1024


In [12]:
# BGE-M3로 유사도 검색
bge_query = bge_embeddings.embed_query("LangChain 에 대해서 알려주세요.")
bge_similarities = np.array(bge_query) @ np.array(bge_embedded_documents).T
bge_sorted_idx = bge_similarities.argsort()[::-1]

print("[Query] LangChain 에 대해서 알려주세요.\n====================================")
for i, idx in enumerate(bge_sorted_idx):
    print(f"[{i}] {texts[idx]}")

[Query] LangChain 에 대해서 알려주세요.
[0] LangChain simplifies the process of building applications with large language models
[1] LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.
[2] 랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. 
[3] 안녕, 만나서 반가워.
[4] Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.


## 4. FlagEmbedding을 활용한 고급 임베딩

FlagEmbedding은 BGE-M3 모델의 고급 기능을 활용할 수 있는 라이브러리입니다. 세 가지 접근법을 제공합니다:

1. **Dense Vector**: 기본적인 dense embedding
2. **Sparse Embedding**: Lexical weight를 활용한 sparse embedding
3. **Multi-Vector (ColBERT)**: 토큰 수준의 세밀한 매칭

### 설치

In [13]:
# FlagEmbedding 설치
!pip install -qU FlagEmbedding

### 4.1 Dense Vector Embedding

In [14]:
from FlagEmbedding import BGEM3FlagModel

model_name = "BAAI/bge-m3"
bge_flagmodel = BGEM3FlagModel(
    model_name, 
    use_fp16=True  # FP16 사용으로 메모리 효율성 향상
)

# Dense embedding 생성
bge_encoded = bge_flagmodel.encode(
    texts,
    batch_size=12,
    max_length=8192,  # 긴 문서 지원
    return_dense=True
)

In [15]:
# 결과 출력(행, 열)
print(f"Dense vector shape: {bge_encoded['dense_vecs'].shape}")

Dense vector shape: (5, 1024)


### 4.2 Sparse Embedding (Lexical Weight)

Sparse embedding은 단어의 중요도를 직접적으로 반영하는 방식입니다.

**특징:**
- 대부분의 값이 0인 고차원 벡터
- 특정 단어나 구문을 정확히 매칭
- TF-IDF나 BM25와 유사한 방식

In [16]:
# Sparse embedding 생성
bge_sparse_encoded = bge_flagmodel.encode(
    texts, 
    return_sparse=True
)

In [17]:
# Lexical matching score 계산
lexical_scores1 = bge_flagmodel.compute_lexical_matching_score(
    bge_sparse_encoded["lexical_weights"][0], 
    bge_sparse_encoded["lexical_weights"][0]
)
lexical_scores2 = bge_flagmodel.compute_lexical_matching_score(
    bge_sparse_encoded["lexical_weights"][0], 
    bge_sparse_encoded["lexical_weights"][1]
)

print(f"Lexical matching score (0 <-> 0): {lexical_scores1}")
print(f"Lexical matching score (0 <-> 1): {lexical_scores2}")

Lexical matching score (0 <-> 0): 1.0
Lexical matching score (0 <-> 1): 0.0


### 4.3 Multi-Vector (ColBERT)

ColBERT(Contextualized Late Interaction over BERT)는 토큰 수준의 세밀한 매칭을 수행합니다.

**작동 방식:**
1. 문서의 각 토큰에 대해 별도의 벡터 생성
2. 쿼리의 각 토큰과 문서의 모든 토큰 간 유사도 계산
3. MaxSim 연산으로 최종 점수 계산

**장점:**
- 토큰 수준의 정밀한 매칭
- 문맥을 고려한 임베딩
- 긴 문서에 효과적

In [18]:
# ColBERT vectors 생성
bge_colbert_encoded = bge_flagmodel.encode(
    texts, 
    return_colbert_vecs=True
)

In [19]:
# ColBERT score 계산
colbert_scores1 = bge_flagmodel.colbert_score(
    bge_colbert_encoded["colbert_vecs"][0], 
    bge_colbert_encoded["colbert_vecs"][0]
)
colbert_scores2 = bge_flagmodel.colbert_score(
    bge_colbert_encoded["colbert_vecs"][0], 
    bge_colbert_encoded["colbert_vecs"][1]
)

print(f"ColBERT score (0 <-> 0): {colbert_scores1:.2f}")
print(f"ColBERT score (0 <-> 1): {colbert_scores2:.2f}")

ColBERT score (0 <-> 0): 1.0
ColBERT score (0 <-> 1): 0.65


## 요약

이 튜토리얼에서는 다양한 HuggingFace 임베딩 방법을 다루었습니다:

1. **HuggingFace Endpoint Embeddings**
   - API를 통한 임베딩 생성
   - 로컬 설치 불필요
   - API 토큰 필요

2. **HuggingFace Local Embeddings**
   - 로컬에서 모델 실행
   - 더 빠른 처리 속도
   - 오프라인 사용 가능

3. **BGE-M3 Embeddings**
   - 우수한 다국어 지원
   - 높은 성능
   - 다양한 디바이스 옵션

4. **FlagEmbedding 고급 기능**
   - Dense Vector: 기본 임베딩
   - Sparse Embedding: 정확한 단어 매칭
   - ColBERT: 토큰 수준 매칭

각 방법은 사용 사례에 따라 장단점이 있으므로, 프로젝트의 요구사항에 맞게 선택하여 사용하시기 바랍니다.